In [1]:
push!(LOAD_PATH, pwd())
include("ClusterIPM.jl")
using StochJuMP
using JuMP
using Distributions
using Ipopt

LoadError: LoadError: LoadError: ArgumentError: StochJuMP not found in path
while loading /home/jordan/git/JuliaBox/IPCluster_examples/ClusterIPM.jl, in expression starting on line 7
while loading In[1], in expression starting on line 2

In [2]:
# Parameters
NS = 10000;                 # number of scenarios
S = collect(1:NS)          # scenario set
P = collect(1:3)           # set of crops (1=wheat,2=corn,3=beets)

prcost = zeros(3)
prcost[1] = 150
prcost[2] = 230
prcost[3] = 260

pcost = zeros(3)
pcost[1] = 238
pcost[2] = 210
pcost[3] = 0

scost = zeros(3)
scost[1] = 170
scost[2] = 150
scost[3] = 36

demand = zeros(3)
demand[1] = 200
demand[2] = 240
demand[3] = 0;

# assign random data
yield = zeros(length(S),3)
yield[S,1] = 2.5
yield[S,2] = 3.0
srand(123)
μ = 20; σ = 5
d = Normal(μ,σ)
yield[S,3] = rand(d,NS)

LoadError: LoadError: UndefVarError: Normal not defined
while loading In[2], in expression starting on line 32

In [3]:
# build the model and solve with IPOPT
m = Model(solver=IpoptSolver())
@variable(m, x[S,P] >= 0)    # acres devoted to crops
@variable(m, y[S,P] >= 0)    # crops purchase
@variable(m, w[S,P] >= 0)    # crops sold;
@variable(m, cost[s in S])
@constraint(m, varcost[s in S], cost[s] == sum{prcost[j]*x[s,j] + pcost[j]*y[s,j] - scost[j]*w[s,j], j in P})
@constraint(m, cap[s in S], sum{x[s,j], j in P} <= 500)
@constraint(m, bal[s in S,j in P], yield[s,j]*x[s,j]+y[s,j]-w[s,j] >= demand[j])
@constraint(m, sellb[s in S], w[s,3] <= 6000)
@constraint(m, buyb[s in S], y[s,3] <= 0)
@constraint(m, nonant[s in S,j in P], x[1,j] == x[s,j])
@objective(m, Min, (1/NS)*sum{cost[s], s in S})
solve(m)

LoadError: LoadError: UndefVarError: IpoptSolver not defined
while loading In[3], in expression starting on line 2

In [4]:
# build with StochJuMP and solve with IPCluster
m = StochasticModel(NS)
@defVar(m, x[P] >= 0)    # acres devoted to crops
@defVar(m, s2 >= 0)
@addConstraint(m, cap, sum{x[j], j in P} + s2 == 500)
@setObjective(m, Min, sum{prcost[j]*x[j], j in P})
for i in 1:NS
    bl = StochasticBlock(m)
    @defVar(bl, y[P] >= 0)    # crops purchase
    @defVar(bl, w[P] >= 0)    # crops sold;
    @defVar(bl, s[P] >= 0)
    @addConstraint(bl, bal[j in P], yield[i,j]*x[j]+y[j]-w[j] - s[j] == demand[j])
    setUpper(w[3], 6000)
    setUpper(y[3], 0)
    @setObjective(bl, Min, 1.0/NS*sum{pcost[j]*y[j] - scost[j]*w[j], j in P})
end
CluIPM_solve(m)

[0/1] create problem 


1

-----------------------------------------------
IP Solver
V.M. Zavala, Argonne National Laboratory, 2012
-----------------------------------------------
  Scenarios ======= 10000
  Variables ======= 90004 | 1st Stage = 4 | 2nd Stage = 90000
  Equality Cons === 30001 | 1st Stage = 1 | 2nd Stage = 30000
  Inequality Cons = 110004 | 1st Stage = 4 | 2nd Stage = 110000
check point 0. Start 
Hello world from process 0 of 1
check point 1. Before calling solver 
iter      objective      inf_pr      inf_du     lg(mu)      alpha     |C|    itla 
   0    0.0000000e+00   5.00e+02   1.37e+07   6.39e+00   0.00e+00       0       0
   1    1.8056443e+05   2.31e+02   3.56e+06   5.84e+00   5.37e-01     100       3
   2    5.3158647e+05   2.07e+02   5.72e+09   5.84e+00   1.06e-01     100       3
   3    5.1394011e+05   2.05e+02   6.30e+09   5.83e+00   9.74e-03     100       3
   4    4.2015405e+05   2.03e+02   1.25e+10   5.89e+00   9.80e-03     100       3
   5    6.2522939e+05   1.60e+01   2.37e+09   5.